<a href="https://colab.research.google.com/github/onni-Q/workshop_notebooks/blob/main/Week4_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important**: Before you run any cells in this notebooks, in the top menu go to **Runtime|Change runtime type** and switch your runtime to **Python 3** with hardware accelerator **T4 GPU**.

When asked, please **allow the notebook access to Google drive**.

Please run the cell below to set up the environment. This take a little while.


**This workshop consists of three parts. The first part covers hate-speech detection and fine-tuning Transformers. The second part 'Case Study: Text Classification Patents' is about patents and text-to-text Transformers, which are the reason why people are now using LLMs also for things like sentiment analysis. The third part 'A Tour of Transformers Applications' is a run through other so-called zero shot applications with Transformers. To get a complete, please complete 2 of the 3 parts.**


In [ ]:
# Keep cell
# Running this takes a while!
%%capture

import sys
import os
import nltk

import matplotlib.pyplot as plt
import torch
import transformers

import pandas as pd
import numpy as np

try:
  import learntools.core
  from learntools.core import binder
  binder.bind(globals())
except:
  !pip install git+https://github.com/Kaggle/learntools.git
  import learntools.core
  from learntools.core import binder
  binder.bind(globals())

if not(os.path.exists('text_workshop_tests.py')):
  print('Downloading text_workshop_tests.py\n')
  !wget https://raw.githubusercontent.com/goto4711/cdai/refs/heads/main/text_workshop_tests.py


from text_workshop_tests import *


#Load models ...

from google.colab import drive
drive.mount('/content/drive')
CACHE_DIR = "/content/drive/MyDrive/CDA-CACHE/huggingface"
os.makedirs(CACHE_DIR, exist_ok=True) # Create the folder if it doesn't exist
os.environ['HF_HOME'] = CACHE_DIR
# Alternatively, for older transformers versions: os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR
print(f"Hugging Face cache directory set to: {os.environ.get('HF_HOME')}")

try:
  import datasets
except:
  !pip install datasets
  import datasets

try:
  import huggingface_hub
except:
  !pip install huggingface_hub
  import huggingface_hub

try:
  import accelerate
except:
  !pip install accelerate
  import accelerate

try:
  import opendatasets
except:
  !pip install opendatasets
  import opendatasets

try:
  import sacremoses
except:
  !pip install sacremoses
  import sacremoses

try:
  nltk.data.find('tokenizers/punkt')
except LookupError:
  nltk.download('punkt')
  nltk.download('punkt_tab')

try:
  nltk.data.find('corpora/stopwords')
except LookupError:
  nltk.download('stopwords')


if not(os.path.exists('TwitterHate_Small.csv')):
  print('Downloading TwitterHate_Small.csv')
  !wget https://raw.githubusercontent.com/goto4711/cdai/main/TwitterHate_Small.csv

if not(os.path.exists('uspppm-data')):
  print('Downloading usppm-data\n')
  !wget https://github.com/goto4711/cdai/raw/main/uspppm-data.zip
  !unzip -q uspppm-data.zip
  !rm uspppm-data.zip


is_colab = "google.colab" in sys.modules
is_kaggle = "kaggle_secrets" in sys.modules
is_gpu_available = torch.cuda.is_available()

def display_library_version(library):
    print(f"Using {library.__name__} v{library.__version__}")

# Give visibility on versions of the core libraries
display_library_version(transformers)
display_library_version(datasets)
# Disable all info / warning messages
transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()
# Logging is only available for the chapters that don't depend on Haystack
if huggingface_hub.__version__ == "0.0.19":
    huggingface_hub.logging.set_verbosity_error()


#hide
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

if not is_gpu_available:
    print('------' * 10)
    print("No GPU was detected! This notebook can be *very* slow without a GPU 🐢")
    if is_colab:
        print("Go to Runtime > Change runtime type and select a GPU hardware accelerator.")
    if is_kaggle:
        print("Go to Settings > Accelerator and select GPU.")

# Cultural Data Forms - Texts

In this workshop, we will continue examining hate-speech detection. As we have seen, our simple methods have failed to deliver convincing results using dictionaries. Transformers (https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)) are the foundation of some of the most popular AI applications like ChatGPT. Will they do better? Don't worry, it is not as complicated as it sounds.

With Transformers, machines can generate text, translate from one language to another, analyze comments, label words in sentences, etc. Transformers have been introduced in a paper called 'Attention is all you need' (https://proceedings.neurips.cc/paper_files/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf). It has become one of the coining papers of the new AI age.

There is a lot of critical work on monopolization in the field of AI, with Open AI owned by Microsoft, who also owns the Cloud infrastructures to run ChatGPT applications, etc. One attempt to work against this is to create open-source versions of deep-learning architectures and models. A leading organisation in this field of open-source AI is Hugging Face, which provides a wide range of models for text analysis or so-called language models (https://huggingface.co/). They make interacting with Transformers quite easy, as you will find out by the end of this workshop. In many ways, the Hugging Face way of doing NLP is easier than what you have seen in the lecture.

By the end of this workshop, you will understand the Hugging Face ecosystem, before you learn how to fine-tune a Transformers model. Then at the end, we find out how easy it all is by using Hugging Face directly without any further training.  The last part 'Tour of Transformer Applications' (zero-shot) shows the breadth of what is possible but you can be selective in what you actually run. If time is tight, you can at first ignore the Case Study on classifying patents.

We can do all the things we introduced in the lecture easier (and often much better) with Transformers from classification to information extraction. Before we reach the easy things, we need to, however, first understand more about the Hugging Face ecosystem. It allows you to load datasets of texts and transform them so that they can be used in deep learning. To this end, it introduces much more efficient tokenizers than we have already seen. Most importantly, Hugging Face's models help us reuse the knowledge stored in other language models. This process is called fine-tuning and means that we can take an existing language model and adjust it to our work. For instance, we could pick up the 'BERT' language model based on the original Transformers paper and fine-tune it so that it would be able to recognize sentiments in the state of the unions.

The figure below shows the Hugging Face pipeline, which we will explore in this workshop. As always, once we have learned the pipeline steps we can build many different applications with it.

<img alt="Hugging Face Pipeline" caption="A typical pipeline for training transformer models with the Datasets, Tokenizers and Transformers" src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg"/>

We will employ Transformers to predict whether an input text is hate speech or not based on examples we will train a model with. Thus, while in the lecture we used unsupervised learning with dictionaries to detect hate speech, we now employ supervised learning. This requires training datasets that are labelled, which we can also get from Kaggle and Hugging Face. To make it more interesting, we switch to more relevant data for this kind of work and look at hate speech in Tweets and social media.

## Case Study: Sentiment analysis of social media

For the sentiment analysis of Tweets, we will use a dataset from the Kaggle platform, which we have already met a couple of times. If you use Kaggle for nothing else in this course, you might find it interesting to look into their datasets. Check out https://www.kaggle.com/datasets to try out your newly gained data skills.

Today, we use a simple dataset for Twitter hate speech https://www.kaggle.com/datasets/prakharprasad/twitter-hate-speech that contains several thousand Tweets and classifies them as either hate speech or not. Please note that we reduce the size of the data to make it easier to analyse in this workshop.

Before we move to our Twitter dataset, let's see how Hugging Face deals with data in general.

### A First Look at Hugging Face Datasets

The Hugging Face data hub contains many datasets (https://huggingface.co/datasets). We can use the `list_datasets()` function from the main Huggging Face Hub library to see which datasets are available.

Run the cell below and ignore the warnings.

In [ ]:
#Keep cell
#Takes a while - please be patient
#Commented out as you might want to skip this cell and the next one. They take a long time and are more for information only.
#Uncomment the code, if you are interested.

from huggingface_hub import list_datasets
# all_datasets = list(list_datasets())

# print(f"There are {len(all_datasets)} datasets currently available on the Hub.")
# print(f"The first 5 are: {all_datasets[:5]}")

We can see that each dataset is given a name/id, so let's see which one looks promising for hate-speech detection. Run the next cell.

In [ ]:
#Keep cell
#Commented out as you might want to skip this cell and the previous one. They take a long time and are more for information only.
#Uncomment the code, if you are interested.

# for s in all_datasets:
#   s_ = os.path.basename(s.id)
#   if "hate" in s_:
#     print(s_)

That's a lot of datasets. There is a lot of research on hate speech. Let's load one of them: `tdavidson/hate_speech_offensive`.

Run the cell below.

In [ ]:
#Keep cell
from datasets import load_dataset
hatespeech_hf = load_dataset("tdavidson/hate_speech_offensive")

Inside `hatespeech_hf`, we find a so-called `DatasetDict`. Print out `hatespeech_hf`.

This Hugging Face creation looks similar to a Python dictionary, with the key of `train` corresponding to a training dataset of over 24k rows at the time of writing. Let's index into `train`. Run
```
train_ds = hatespeech_hf["train"]
train_ds
```

Let's run a test that everything is set up ok.

In [ ]:
# Run this cell to test your code...
assert len(train_ds) > 24760, f'Test failed'
print('All tests passed!')

Now, we have our `Dataset` object as the core structure in datasets. Among other things, it behaves like a Python list. Try and run `len` with `train_ds`.

We can also access a single example by its index. Type in `train_ds[0]`.

The documentation for this dataset can be found at https://huggingface.co/datasets/tdavidson/hate_speech_offensive/. Thus the above keys indicate:

- count: (Integer) number of users who labelled each Tweet - min is 3, sometimes more users coded a Tweet when judgements were determined to be unreliable.
- hate_speech_annotation: (Integer) number of users who judged the Tweet to be hate speech.
- offensive_language_annotation: (Integer) number of users who judged the Tweet to be offensive.
- neither_annotation: (Integer) number of users who judged the Tweet to be neither offensive nor non-offensive.
- label: (Class Label) class label for majority of CF users (0: 'hate-speech', 1: 'offensive-language' or 2: 'neither').
- tweet: (string).

Run `train_ds.column_names`.

The data types for each column can be found in the `features` attribute of `train_ds`. Do you know how to print it out? A quick online search will also help.

In [ ]:
q0.hint()
#q0.solution()

So, `count`, for instance, is an integer, while `tweet` is a string. Let's print out the first three elements with `train_ds[:3]`.

The dictionary values are lists instead of individual elements.

We can also go to the full key by name. The first five Tweets can be printed with `train_ds["tweet"][:5]`. Try it.

### What if my dataset is not on the hub?

Most often, a dataset will not be on the hub. The hate speech dataset is actually a version of a famous dataset on Kaggle, as far as I can see https://www.kaggle.com/code/prakharprasad/twitter-hate-speech-classification. Somebody has imported it to Hugging Face.

Let's look at the simpler hate-speech dataset at https://www.kaggle.com/datasets/prakharprasad/twitter-hate-speech to understand how to import it into our processing with Hugging Face. We have downloaded a smaller version of it for you to make a bit to make it more balanced and easier to process with Hugging Face's dataset tools: https://huggingface.co/docs/datasets/en/process. Label is either 1 for a Tweet containing hate speech or 0 otherwise. 'tweet' contains the Tweet.

We can load the local dataset with `load_dataset` by using the `csv` script and pointing the `data_files` argument to `"TwitterHate.csv"`. The separator is '\t' for tab-delimited and names are the column names. `class_encode_column('label')` tells the Transformers to use 'label' as a target column.

Run the next cell.

In [ ]:
#Keep cell
ds = load_dataset("csv", data_files="TwitterHate_Small.csv")
ds = ds.class_encode_column('label')

The `load_dataset()` function is very powerful: https://huggingface.co/docs/datasets/loading.html. Let's take a look at the data next and print out `ds` and some data (the first 5 entries). Complete the cell below.

In [ ]:
#Complete cell
print(ds)
print(___['train'][:5])

In [ ]:
q1.hint()
#q1.solution()

As we are experts in machine learning by now, we know that we need a test dataset next to the training dataset. Actually, we would have also needed a validation dataset to make sure that our algorithm is good. It allows us to use the test data only for a final test and tune the model with the validation set. But we skip this here for learning reasons. You can read up on this at https://en.wikipedia.org/wiki/Training,_validation,_and_test_data_sets.

For the training and test data, we choose an 75-25 random split by running the cell below. Do you see how? Do you remember how we did this for sklearn? For the Hugging Face version with its `train_test_split()`, run the next cell.

In [ ]:
#Keep cell
hatespeech_local = ds['train']
hatespeech_local = hatespeech_local.train_test_split(test_size=0.2, seed = 42)
hatespeech_local

### From Datasets to DataFrames

Pandas is great for analyzing data and visualizing it. We can convert a `Dataset` to a Pandas `DataFrame` with the `set_format()` method. Run the cell below.

In [ ]:
#Keep cell
import pandas as pd
import numpy as np

hatespeech_local.set_format(type="pandas")
df_hs = hatespeech_local["train"][:]
df_hs.head()

Let's run a quick check whether everything is loaded.

In [ ]:
# Run this cell to test your code...
assert df_hs.shape[0] == 6548, f'Test failed'
print('All tests passed!')

As a reminder, label 0 stands for non-hateful and 1 for hateful. Let's add a column 'label_name' to reflect this. Do you remember `np.where`? Complete the cell below.

In [ ]:
#Complete cell
df_hs['label_name'] = np.where(df_hs['label'] == ___, 'Non-hateful', 'Hateful')
df_hs.head()

In [ ]:
q2.check()
#q2.hint()
#q2.solution()

Whenever you are working with NLP classifications or classifications in general, it is good to start with understanding the distribution across the target classes. We cannot cover this here, but skewed classes have to be addressed in very different ways from more even distributions.

Let's visualize `label_name`, which is our target variable. Complete the cell below.

In [ ]:
#Complete cell

import matplotlib.pyplot as ___

df_hs["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.___()

In [ ]:
q3.hint()
#q3.solution()

In this case, we can see that the dataset is heavily imbalanced. We could try and find more hateful Tweet examples, but we skip this here. Instead let's look how long the Tweets are. This is important for several reasons. We have already discussed the impact of text lengths in the lecture but for language models there is also an important additional reason. Transformer models have a maximum input sequence length that is referred to as the _maximum context size_. This is what they can see from a text at any particular moment in time, because of the way that they have been trained.

Let's look at a leaner version of the already mentioned BERT language model called DistilBERT (https://huggingface.co/docs/transformers/en/model_doc/distilbert). According to this documentation, its maximum context size is 512 tokens, which amounts to a few paragraphs of text. This should be fine for Tweets but let's check. Complete the cell below to visualise the Tweet lengths.

In [ ]:
#Complete cell

df_hs["Words Per Tweet"] = df_hs[___].str.split().apply(len)
df_hs.boxplot("Words Per Tweet", by="label_name", grid=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
q4.hint()
#q4.solution()

Most Tweets are around 15 words long and even the longest Tweets are well below DistilBERT's maximum context size. If that would not be the case, we have to apply specific methods to reduce the lengths of the input texts. Or we 'chunk' the texts into smaller parts: https://cobusgreyling.medium.com/chunking-for-llms-using-haystack-huggingface-81255776d71.

Before we do anything else, we have to reset the output format of our dataset since we don't need the dataframe-format anymore. Run `hatespeech_local.reset_format()`.

In [ ]:
hatespeech_local.reset_format()

We have already talked about tokens/tokenization a bit in the lecture but what are they in the Transformers world? Let's find out next.

## From Text to Tokens

As said, tokenization is the second important step for Transformers after loading the datasets. Models like DistilBERT cannot process strings but need  _tokenized_ texts that are _encoded_ as numerical vectors. Tokenization is the step of breaking down a string into the atomic units used in the model. There are several tokenization strategies one can adopt, and the optimal splitting of words into sub-units is learned from the corpus nowadays.

Let's consider the two cases, which have been most common before Transformers: _character_ and _word_ tokenization.

### Character Tokenization

The simplest tokenization scheme is to feed each character individually to the model. In Python, `str` objects are implemented as collections of characters, which allows us to quickly implement character-level tokenization with just one line of code. Run the cell below.

In [ ]:
#Keep cell
text = "Tokenizing text is fun but also strange. Even poetry becomes a number!"
tokenized_text = list(text)
print(tokenized_text)

Afterwards, we need to convert each character into an integer, a process  called _numericalization_. Deep learning/transformers requires all input to be numbers. Numericalisation is easy for characters by simply assigning them a unique integer. The code below is typical for Transformers. It creates a dictionary, which maps tokens (here characters) to numbers by assigning an integer to them - in the order of appearance of the tokens. Run the cell.

In [ ]:
#Keep cell
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

We can now use `token2idx` to transform the tokenized text to a list of integers. Complete the cell below.

In [ ]:
#Complete cell
input_ids = [token2idx[token] for token in ___]
print(input_ids)

In [ ]:
q5.check()
#q5.hint()
#q5.solution()

With a bit more processing, we could use these numbers based on characters for the deep-learning model. But we move on and look at word tokenization.

### Word Tokenization

Instead of splitting the text into characters, we can split it into words, as we have seen in the lecture, and map each word to an integer. One simple class of word tokenizers, we have already seen, uses whitespaces to tokenize the text. We can do this by applying Python's `split()` function directly on the raw text - just like we did to measure the Tweet lengths.

Run the cell below.

In [ ]:
#Keep cell
tokenized_text = text.split()
print(tokenized_text)

There are several problems with this approach. Most importantly maybe, it does not consider the position of a word and its context. Furthermore, the resulting vocabulary can be very large very quickly. This requires very big neural nets and makes their training very expensive.

A common mitigation approach is to limit the vocabulary and discard rarer words and only keep, say, the 100,000 most common words. Words that are not part of the limited vocabulary are classified as 'unknown' and mapped to a shared `UNK` token. You will encounter this approach quite frequently, but this also means that we lose some potentially important information in the process of word tokenization.

Wouldn't it be nice if there was a compromise between character and word tokenization that preserves all the input information _and_ some of the input structure? _subword tokenization_ addresses this.

### Subword Tokenization

The basic idea behind subword tokenization is to combine the best aspects of character and word tokenization. On the one hand, we want to split rare words into smaller units to allow the model to deal with complex words and misspellings. On the other hand, we want to keep frequent words as unique entities so that we can keep the length of our inputs to a manageable size.

The main distinguishing feature of subword tokenization (as well as many forms of modern word tokenization) is that it is _learned_ from the pretraining corpus using a mix of statistical rules and algorithms, similar to what we have seen for Word2Vec in the lecture. Subword tokenization is a very active research field in computer science. Here, we only focus on its application, which is very easy with Hugging Face.

The subword tokenizer WordPiece (https://ieeexplore.ieee.org/document/6289079) is used by BERT and DistilBERT. The easiest way to understand how WordPiece works is to observe it in action. `transformers` provides a convenient `AutoTokenizer` class that allows you to quickly auto-load the tokenizer associated with a model. We just call the `from_pretrained()` method. We also define a so-called 'checkpoint', which one can think of as the latest save of the model. In this case, it is the latest version of the DistilBERT model https://huggingface.co/distilbert/distilbert-base-cased. This version of DistilBERT makes no difference between lower and upper cases in English.

Run the cell below.

In [ ]:
#Keep cell
from transformers import AutoTokenizer

checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The `AutoTokenizer` class belongs to a larger set of so-called 'auto' classes (https://huggingface.co/docs/transformers/model_doc/auto), whose job it is to automatically retrieve a model's configuration,  weights, etc. from its latest checkpoint. This allows you to quickly switch between models, but you can also load the specific model manually.

At the beginning of your learning journey, it is best to rely as much on 'auto' classes as possible. Later on, you will see that we could have also loaded the DistilBERT tokenizer directly. For instance, you could type in:
```
from transformers import DistilBertTokenizer

distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
```
Check it out online if you want but we move on for now.

We want to see `tokenizer` in action with our earlier defined `text`. Complete the cell below.

In [ ]:
#Complete cell
encoded_text = ___(text)
print(encoded_text)

In [ ]:
q6.hint()
#q6.solution()

Every subword has now a unique integer in the `input_ids` field. You can ignore `attention_mask`. With `input_ids`, we can convert them back into tokens by using the tokenizer's `convert_ids_to_tokens()` method. Run the next cell.

In [ ]:
#Keep cell
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

Let's run a quick test that all is good.

In [ ]:
# Run this cell to test your code...
assert tokens[-1]=='[SEP]', f'Test failed'
print('All tests passed!')

From the output, we can observe three things:
1. Some special tokens, `[CLS]` and `[SEP]`, have been added to the start and end of the sequence. These tokens differ from model to model, but their main role is to indicate the start and end of a sequence.
1. The tokens have each been lowercased, which is a feature of this version of DistilBERT.
1. Finally, we can see that the word 'tokenizing', e.g., has been split into two tokens, since it is not a common word. The `##` prefix in `##izing` means that the preceding string is not a whitespace. With the prefix, we can convert them back.

The `AutoTokenizer` class has a `convert_tokens_to_string()` method for converting tokens back. Complete the next cell.

In [ ]:
#Complete cell
tokenizer.convert_tokens_to_string(___)

In [ ]:
q7.hint()
#q7.solution()

The `AutoTokenizer` class also has several attributes that provide information about the tokenizer. For example, we can inspect the vocabulary size or number of tokens with `tokenizer.vocab_size`. `vocab_size` is an attribute of `tokenizer` and `.` gives you access to these. Try to print out `tokenizer.vocab_size`.

The model's maximum context size or its maximum number of tokens is provided by `tokenizer.model_max_length`.

With this basic understanding of the tokenization process, let's see how we can tokenize the whole dataset.

### Tokenizing the Whole Dataset

To tokenize the whole dataset, we will use the `map()` method of our `DatasetDict` object. To this end, we need a processing function to tokenize our examples. Run the cell below to define this function.

In [ ]:
#Keep cell
def tokenize(batch):
    return tokenizer(batch["tweet"], padding=True, truncation=True)

This function applies the tokenizer to a batch of examples; `padding=True` will pad the examples with zeros to the size of the longest one in a batch, and `truncation=True` will truncate the examples to the model's maximum context size. Just ignore these for the time being. Always, set them to True if you create your own tokenizer at the beginning.

Creating your own tokenizer is simple using this example code. You just have to define the `tokenizer` function in the example above for your own Tokenizer.

To see `tokenize()` in action, let's pass a batch of two examples from the training set. Run the cell below.

In [ ]:
#Keep cell
tokenize(hatespeech_local["train"][:2])

Here we can see the result of padding. The second example is shorter than the first. So, zeros have been added to that element to make 'pad' them to the same length. Again, ignore `attention_mask`.

The zeros have a corresponding `[PAD]` token in the vocabulary as a special token. The set of special tokens also includes the `[CLS]` and `[SEP]` tokens that we encountered earlier. Run the cell below to see the IDs of special tokens.

In [ ]:
#Keep cell
tokens2ids = list(zip(tokenizer.all_special_tokens, tokenizer.all_special_ids))
data = sorted(tokens2ids, key=lambda x : x[-1])
pd.DataFrame(data, columns=["Special Token", "Special Token ID"]).T

With the tokenization function `tokenize(batch)`, we can apply it across all the splits in the dataset very easily with `map()`. We print the column_names of 'test' to demonstrate our success. Run:
```
hatespeech_encoded = hatespeech_local.map(tokenize, batched=True, batch_size=None)
print(hatespeech_encoded["test"].column_names)
```

Hugging Face's Transformers assume that your target variable has the column name 'labels' and the textual input is 'text'. Therefore, we need to rename the columns with `rename_columns()`. Run the cell below.

In [ ]:
#Keep cell
hatespeech_encoded = hatespeech_encoded.rename_columns({'label':'labels', 'tweet': 'text'})
print(hatespeech_encoded["test"].column_names)

This is just an exercise. So, let us reduce the training and test datasets, which is surprisingly hard with Hugging Face at the time of writing. The cell below takes care of that, by picking up 50% of the datasets. It is a bit complicated but a good exercise to try and understand the various parts. Also, it might be useful for the future when you are working with your own datasets.

Run the cell.

In [ ]:
#Keep cell
frac_ = 0.5

train_frac = int(hatespeech_encoded["train"].num_rows * frac_)
test_frac = int(hatespeech_encoded["test"].num_rows * frac_)

hatespeech_encoded_train_small = hatespeech_encoded["train"].shuffle(seed=42).select(range(train_frac))
hatespeech_encoded_test_small = hatespeech_encoded["test"].shuffle(seed=42).select(range(test_frac))

Let's print out `hatespeech_encoded_train_small` and `hatespeech_encoded_test_small` to check that everything looks alright.

## Transformer Models

DistilBERT is pretrained to predict masked words in a sequence of text. Let's say we have a sentence 'The house has a garden.' Then, house might be hidden with 'The ___ has a garden.' DistilBERT learns that 'house' has likely been hidden. This learning task allows the model to collect a lot of information on languages in general, but not on the particular text classification task like hate-speech detection. We need to modify the baseline language models. The next step is then to 'fine-tune' a model like DistilBERT for a particular task like in our case hate-speech analysis. Do you remember the lecture?

### Baseline

Before we investigate fine-tuning we want to quickly run a baseline model to compare our Transformer results to. In our case, this will be the dictionary-based one that we have already discussed. Run the next cell to add a column of hate_score to `df_hs`. Note that this is still a sample and not the full dataset.

Do you remember the code from the lecture? The cell puts it all together.

In [ ]:
# Keep cell
# Repeat dictionary-based sentiment analysis

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords_ = list(stopwords.words('english'))
neg_words = list(set(list(pd.read_csv('https://raw.githubusercontent.com/goto4711/social-cultural-analytics/master/hate-vocab-eng.csv')['word'])))

def hate_score_(text):
    score = 0
    words = [word.lower() for word in word_tokenize(text)]
    for word in words:
        if word in neg_words:
            score += 1
    return score

df_hs = df_hs.sample(frac=frac_, random_state=1)

df_hs['hate_score'] = df_hs['tweet'].apply(hate_score_)
df_hs['hate_predict'] = np.where((df_hs['hate_score'] > 0), 'Hateful', 'Non-hateful')

df_hs["hate_predict"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

`hate_score_` counts the number of times a word from the hatespeech dictionary appears in the tweets. It assigns that count to `df_hs['hate_score']`. If the hate_score is larger than 0, the Tweet is predicted to be 'Hateful' and otherwise 'Non-hateful'.

Let's calculate the accuracy:

In [ ]:
#Keep cell
from sklearn.metrics import accuracy_score

accuracy_score(df_hs["label_name"], df_hs["hate_predict"])

The accuracy looks ok and much better than guessing either of the two labels, which would be 0.5 However, this is mainly because the dominant 'Non-hateful' class is predicted correctly, as the confusion matrix shows. Run the cell below.

In [ ]:
#Keep cell
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(df_hs["label_name"], df_hs["hate_predict"], labels=["Non-hateful", "Hateful"])
pd.DataFrame(cm, columns = ["Non-hateful", "Hateful"], index = ["Non-hateful", "Hateful"])

The model pretty much fails to predict hateful Tweets and is not useful in practice. You should be very sceptical if companies just present basic accuracy to praise their models.

At least we now have a baseline of accuracy we aim to improve by fine-tuning pretrained Transformer models. Before we do this, we have to run the slightly strange looking code below, which runs the model on a GPU (or MPS for Macs) if available rather than a CPU. This makes running Transformers much faster. GPUs are the basic hardware language modelling uses to efficiently calculate billions of operations. If you are interested in the different options check out https://raoofnaushad7.medium.com/difference-between-cuda-and-mps-3d2596629c89.

Run the code below to set the default device to run our model on as either CUDA(GPU), MPS or CPU.

In [ ]:
#Keep cell

if torch.cuda.is_available():
    DEFAULT_DEVICE = "cuda"
else:
    if torch.backends.mps.is_available():
        DEFAULT_DEVICE = "mps"
    else:
        DEFAULT_DEVICE = "cpu"
device = torch.device(DEFAULT_DEVICE)
print(DEFAULT_DEVICE)

It should print out the device you are using. In my case it would be 'mps' for my home computer and 'cuda' on Google Colab if you connected the runtime correctly. If you see 'cpu', start from the beginning, connect the right runtime and run all cells above again.

Let's go on and finally fine-tune.

### Fine-Tuning Transformers

We want to fine-tune the pretrained DistilBERT model and first need to get it from `AutoModelForSequenceClassification`. The Hugging Face `AutoModelForSequenceClassification` model has a so-called classification _head_ on top of the pretrained model body, which can be trained. We just need to specify how many labels the model has to predict since this dictates the number of outputs the classification head has. We set `num_labels=2` for the binary choice of hate_speech or not.

Run the cell below. The most important line is `model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)`, which uses the `checkpoint = "distilbert-base-cased"` from the tokenizer above to load `AutoModelForSequenceClassification` for a binary classification (`num_labels=2`). `to(device)` sends the calculations to our default device, which should be cuda on Colab.

In [ ]:
#Keep cell
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

You might see some warnings in the following code about Hugging Face versions. Please ignore these for this session.  

The next step is to define the metrics that we'll use to evaluate our model's performance during fine-tuning. To this end, Hugging Face requires a `compute_metrics()` function for its `Trainer` that we will define later.  This function receives a `pred`, which is a  tuple with `predictions` and `label_ids` attributes. It has to return a dictionary that maps each metric's name to its value.

For our classification application, we will compute the accuracy of the model, as this allows us to compare with our baseline. Try and understand the cell below and run it. You can reuse this function or a similar one for many Hugging Face classifications. Just keep it in mind.

Run the cell below.

In [ ]:
#Keep cell
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

### Training the model

To run the Hugging Face `trainer` (https://huggingface.co/docs/transformers/main_classes/trainer), we first define some of its parameters with `TrainingArguments`. It stores a lot of information and gives you fine-grained control over training and evaluation. You can ignore most of the arguments for now, as they are used for reporting only. Just copy and paste `TrainingArguments` next time you want to use it.

The most important argument  is `output_dir`, which is where all the files from training are stored. Three more parameters are important. `batch_size` defines the number of examples to work through before updating internal model parameters. It has a lot to do with what your machine can process in its memory at any moment in time. At the beginning, it is probably a good idea not to go beyond 128 and if you have problems use 64 or 32 as batch size. You computer will complain a lot if you get the batch size wrong. Thus, if you see strange memory errors, a good idea is always to first reduce the batch size. This generally leads to longer processing times though, because fewer records are processed at the same pass. `epochs` are the number of passes through the network. The larger the number of epochs the higher the chances for the network to learn, but it also takes much longer. For demonstration purposes, we set `epochs = 2`. Finally, `learning_rate=2e-5` sets the speed of updating parameters in deep-learning models.

Run the cell below.

In [ ]:
#Keep cell
from transformers import TrainingArguments

batch_size = 128
epochs = 2

training_args = TrainingArguments(output_dir=f"{checkpoint}-finetuned-emotion",
                                  num_train_epochs=epochs,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  eval_strategy="epoch",
                                  disable_tqdm=False,
                                  report_to="none")


We can can now fine-tune our model with the `Trainer`. Check the documentation for the explanation of the arguments.

Run the next cell. This might take a while as you are now doing really advanced stuff.

In [ ]:
#Keep cell
from transformers import Trainer

trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=hatespeech_encoded_train_small,
                  eval_dataset=hatespeech_encoded_test_small,
                  processing_class=tokenizer)
trainer.train();

Looking at the logs, we can see that our model has an accuracy on the test dataset of > 88%, which is a great improvement on our baseline dictionary-based method.

To visualize the confusion matrix next, we first need to get the predictions on the validation set. The `predict()` method of the `Trainer` class returns several useful objects for evaluation. Run the cell below.

In [ ]:
#Keep cell
preds_output = trainer.predict(hatespeech_encoded_test_small)

Run another test that you are on the right way.

In [ ]:
# Run this cell to test your code...
assert len(preds_output) == 3, f'Test failed'
print('All tests passed!')

The output of the `predict()` contains arrays of `predictions` and `label_ids`, along with the metrics we passed to the trainer. For example, the metrics can be accessed with `preds_output.metrics`. Try it.

`preds_output` also contains the raw predictions for each class. A single prediction has one value for each class: 'Non-hateful' and 'Hateful'. Take a look at the first prediction by running the cell below.

In [ ]:
#Keep cell
preds_output.predictions[0]

You will remember the outputs of classifications we have seen before? As always we get two outputs, one for each class, and we simply choose the larger one.

We can decode the predictions and select always the most likely class by using `np.argmax()`. `np.argmax` returns the indices of the maximum values. So in my output, it would return 1 has the 1 index is larger than the 0 index. In your case, this might look differently.

In my case, 1 rightly predicts a 'hateful' Tweet. We can run `np.argmax()` against all the predictions at the same time with `y_preds = np.argmax(preds_output.predictions, axis=1)`. `axis=1` indicates a row-wide comparison.



Print out the first ten y_preds next.

In [ ]:
q8.check()
#q8.hint()
#q8.solution()

Run another test.

In [ ]:
# Run this cell to test your code...
assert len(y_preds) > 810, f'Test failed'
print('All tests passed!')

With the predictions, we can plot the confusion matrix. Complete the cell below by looking back at the earlier example or using the documentation for `confusion_matrix`.

In [ ]:
#Complete cell
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test = np.array(hatespeech_encoded_test_small["labels"])

cm = ___(y_test, y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=___)
disp.plot()
plt.show()

In [ ]:
q9.hint()
#q9.solution()

This is much better. Now, the model is actually able to predict 'hateful' Tweets (label=1).

We can also run our own Tweet examples by using Hugging Face's pipeline. This is also called inference for language models.

Run the cell below to define `pipe` and set up a `TextClassificationPipeline`. A Hugging Face `pipeline` (https://huggingface.co/docs/transformers/en/main_classes/pipelines) allows us to bring together several steps. In this case, we 'pipe' the tokenizer into the model. Please, observe that we have to move the model back to the 'cpu' first. Otherwise, the tokenizer and model are on different devices. Can you identify in the code how we do this? It's easy.

In [ ]:
#Keep cell
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model.to('cpu'), tokenizer=tokenizer)

Let's test the pipeline with two example Tweets, taken from my test data. Run the cell below.

In [ ]:
#Keep cell
nice_tweet = "'i am looking forward to seeing it!! @user @user'"
print('Nice Tweet: ', pipe(nice_tweet, top_k=None))

bad_tweet = "black genius v white supremacy are you missing the action? the #freemasons being taught a lesson about .'."
print('Bad Tweet: ', pipe(bad_tweet, top_k=None))

As you can see, the model correctly predicts non-hateful content for `nice_tweet` with a higher score for 'LABEL_0'. It is also right for the `bad_tweet`. Try some other Tweet texts!

Congratulations, you now know how to train a Transformer model to classify Tweets. This can only be the beginning but hopefully you found it interesting. The most important lesson to remember are the processing steps from the Hugging Face figure above, which are very standard and repeat themselves. The steps are also repeated in the first chapter of the Transformers book with some added extra explanations: _Lewis Tunstall, Leandro von Werra, Thomas Wolf (2022): Natural Language Processing with Transformers, O'Reilly Media, Inc. While inspired by the book, we have changed the datasets, adjusted the task and removed some additional content. But the book is a great introduction to NLP with Transformers written by the Hugging Face team. It is, however, quickly getting outdated by changes to the Hugging Face libraries.

Does this look scary? Maybe, but it is actually always the same steps. Why not ask Gemini whether it can do. This time, do not use a code cell directly but click on Gemini symbol. A chat interface opens. Ask Gemini to "classify from ds the label from tweet using HuggingFace's DestilBERT". It should print out the same steps we have just gone through and create the code for you. I just had to make a few simple adjustments.

The next example is taken from another wonderful resource: the FastAI training course. The patents' case study repeats once more the steps how to use Transformers for text analysis. The corresponding book is _@Howard, J. and Gugger, S. (2020): Deep Learning for Coders with Fastai and Pytorch: AI Applications Without a PhD, O'Reilly Media, Inc._ It is open access at https://www.fast.ai/.

Let's take a look at their example for text classification, which introduces another twist showcasing the power of language models. Before this new twist, many NLP tasks were tackled with specialized models, as we have seen. For example:

- Classification tasks (like sentiment analysis) used models that output class labels.
- Translation models outputted target-language sentences.


Now, we can use only LLMs, their training procedure and code, for a wide range of NLP problems by simply saying that all these different things can be mapped towards text-to-text input-outputs. Check out this famous paper that summarises the idea: https://arxiv.org/pdf/1910.10683.

## Case Study: Text Classification Patents

In my research, I have used text analysis extensively to analyse Tweets. But I have also been interested in patents to understand corporate practices in relation to deep learning, which has inspired me to this case study. We will analyse the Kaggle U.S. Patent Phrase to Phrase Matching competition from https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/. It is described as matching different phrases:

"In this competition, you will train your models on a novel semantic similarity dataset to extract relevant information by matching key phrases in patent documents. Determining the semantic similarity between phrases is critically important during the patent search and examination process to determine if an invention has been described before. For example, if one invention claims "television set" and a prior publication describes "TV set", a model would ideally recognize these are the same and assist a patent attorney or examiner in retrieving relevant documents. This extends beyond paraphrase identification; if one invention claims a "strong material" and another uses "steel", that may also be a match. What counts as a "strong material" varies per domain (it may be steel in one domain and ripstop fabric in another, but you wouldn't want your parachute made of steel). We have included the Cooperative Patent Classification as the technical domain context as an additional feature to help you disambiguate these situations."

Thus, we want to compare two words or short phrases in order to score them whether they are similar or not in a particular context, based on which patent class they were used in. With a score of 1, the two inputs are considered to have identical meaning, while 0 means that they have totally different meaning. 'Abatement' (https://www.dictionary.com/browse/abatement) and 'eliminating process', e.g., have a score of 0.5, meaning they are somewhat similar but not identical. I have to trust that this is right, as they are specific patent ideas with their own meanings, which I know very little about. You will meet a couple of these in this case study.

### Preparing the data

The first step to prepare the data is to download it from Kaggle. If you want to do this by yourself, you need a free account with Kaggle. Please go and register with Kaggle (https://www.kaggle.com/discussions/getting-started/45113).

The library `opendatasets` takes care of all the issues with downloading datasets from online sources with authentication. Uncommenting the line `od.download('https://www.kaggle.com/datasets/atharvaingle/uspppm-data')` and running it, might prompt you for your username and password to access Kaggle. You also need a so-called API key, which can be difficult to find (https://christianjmills.com/posts/kaggle-obtain-api-key-tutorial/). They will be covered again later in the course. If you have problems, come and talk to us.

In this case, you should anyway not have to download any data, as we have done this already for you. But it is good to learn about downloading data from Kaggle for the future.

Run the cell below.

In [ ]:
# Keep cell
# Download to folder ./uspppm-data
import opendatasets as od

if os.path.exists('./uspppm-data'):
    print('Data already downloaded')
else:
    print('Make sure to download the data by running the set-up cells or run the next line to connect to Kaggle.')
    #od.download('https://www.kaggle.com/datasets/atharvaingle/uspppm-data')


Let's load the data into a dataframe `df_pat`. Complete the cell below and print out `df_pat.info()` and `df_pat.head(3)`.

In [ ]:
#Complete cell

df_pat = pd.read_csv(___)
print(___)
print(___)

In [ ]:
q10.check()
#q10.hint()
#q10.solution()


The data looks quite strange, but it is explained at https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data. The  columns stand for:
- id - a unique identifier for a pair of phrases.
- anchor - the first phrase.
- target - the second phrase.
- context - the CPC classification (version 2021.05), which indicates the subject within which the similarity is to be scored.
- score - the similarity, which is sourced from a combination of one or more manual expert ratings.

Let's run a quick test that everything is loaded ok.

In [ ]:
# Run this cell to test your code...
assert df_pat.shape[0] == 36473, f'Test failed'
print('All tests passed!')

By typing in `df_pat['anchor'].value_counts()`, we see that there is a wide variety of first phrases.

The similarity matching we are looking for is made more difficult by the fact that it is dependent on the context of the patent class or CPC classification. Can you change the last expression so that we get the counts for `df_pat['context']`?

In [ ]:
q11.hint()
#q11.solution()

Finally, let's look at our target variable, which is `score`. Run `value_counts()` against the target.

Again, we find a large variety of different scores. Later, we will simplify them, but let's first take a look at examples of each of them. Run the cell below, which is also an example of the power of `print()` by making heavy use of the f-strings we met at the beginning of the course: https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/. It's very useful knowledge from now on as we are dealing with a lot of transformations of text.

In [ ]:
#Keep cell

def print_relation(i):
    return f"WITHIN {df_pat['context'][i]}, THE PHRASE {df_pat['target'][i]} AND THE PHRASE {df_pat['anchor'][i]} HAVE SIMILARITY {df_pat['score'][i]}"

print(print_relation(311))
print(print_relation(1))
print(print_relation(100))
print(print_relation(1500))
print(print_relation(35985))

How can we transform this data into something a language model can classify? We need to get creative and join various columns to create the input. We can then  rely on the language model to understand what it is supposed to do. This is a very elegant way of addressing the problem - also in comparison to the way we defined a classification task earlier. Compared to earlier, we just set up the language model in the right way - with the right kind of text for input and output - and do not need to add complicated programming techniques like particular models. This is why I have added this example to showcase the power of language modelling.

It has become a  text analytics skill to creatively format diverse inputs for a text-to-text model. People like to use very large language model to do specific tasks but then they need to speak to them in a text/language they understand. But this section also reiterates the tokenization and training steps to make you more familiar with the pattern. We will skip some details as the baseline comparison, as the goal is to show the process.

Let's take it one step a time and look at the first example. To a language model, the relation between 'absorption of acid' and 'acid absorption' within the context 'B08' can be represented using a joined text like (in a simplified notation):

```
'CON: ' + df_pat['context'] + '; PHARSE1: ' + df_pat['target'] + '; PHARSE2: ' + df_pat['anchor']
```

As you can see, we have added the columns to the text and concatenated all the important columns with `+`. Now the task is for the model to match these inputs to the target score in order to address the Patent Phrase Matching problem. Run:

```df_pat['input'] = 'CON: ' + df_pat['context'] + '; PHARSE1: ' + df_pat['target'] + '; PHARSE2: ' + df_pat['anchor']```.

For `df_pat`, we use `+` to concatenate the relevant columns into a new string and store it in the new column `input`. We cannot use f-strings directly on a Pandas series.

Next, as promised, let's simplify the classification to a binary one.

Looking back at the output of `df_pat['score'].value_counts()`, a good split seems to be right in the middle. Please run `df_pat['score'] = np.where(df_pat['score']  < 0.5, 0, 1)`. Hopefully, you remember this step from our earlier examples like the music-quality classification.

Print out the first three rows of `df_pat` to check that everything is as desired.

Let's also run a quick test to see that everything is ok.

In [ ]:
# Run this cell to test your code...
assert df_pat.loc[100, 'input'] == 'CON: E03; PHARSE1: abnormal; PHARSE2: abnormal position', f'Test failed'
print('All tests passed!')

Looks good. This is it in terms of initial data preparation. We are ready to tokenize next.

### Tokenization

This is also a good time to demonstrate how to create a Transformer dataset from a dataframe like `df_pad`. It's easy. Run the cell below.

In [ ]:
#Keep cell

from datasets import Dataset, DatasetDict
ds_pat = Dataset.from_pandas(df_pat)
ds_pat

Our next step in the Hugging Face workflow is to pick a model from the hub for tokenization and prediction. Let's use the one that Fastai recommends: https://huggingface.co/microsoft/deberta-v3-small.

Run the cell below to get the checkpoint.

In [ ]:
#Keep cell

checkpoint_pat = 'microsoft/deberta-v3-small'

Do you see how the steps with Transformers are repeating themselves?

The next step is to call AutoTokenizer and create a tokenizer appropriate for a given model. Complete the cell below by adding our model's checkpoint.

In [ ]:
#Complete cell by adding the model's checkpoint

tokenizer_pat = AutoTokenizer.from_pretrained(___)

In [ ]:
q12.hint()
#q12.solution()

A quick example shows us that we are working again with a subword tokenizer. Run the cell.

In [ ]:
#Keep cell

tokenizer_pat.tokenize("Hello Amsterdam, I'm in a lovely city.")

Looking back at the pipeline steps above, the next step is to define a function to tokenize all our inputs, which we call `tokenizer_pat_func`. Run
```
def tokenizer_pat_func(batch):
    return tokenizer_pat(batch["input"])
```

To run this quickly and in parallel on every row, you remember that you can use `map()`. Complete the cell below.

In [ ]:
#Complete cell
ds_pat_encoded = ds_pat.___(tokenizer_pat_func, batched=True)
ds_pat_encoded

In [ ]:
q13.hint()
#q13.solution()

As you can see, this adds a couple of encoded columns with the extension `_ids`.

Let's take a look at the encoded version of `input` for the first row of our data. Complete the cell below.

In [ ]:
#Complete cell

print('input: ', ds_pat_encoded[0]['input'])
print('input_ids: ', ds_pat_encoded[0]['input_ids'])

In [ ]:
q14.hint()
#q14.solution()

Running `tokenizer_pat_func()`, we have created a vocabulary (`vocab`), which contains a unique integer for every possible token string.

E.g., to access the id of the word 'instance' we run `tokenizer_pat.vocab['instance']`.

Running a test next ...

In [ ]:
# Run this cell to test your code...
assert tokenizer_pat.vocab['house'] == tokenizer_pat.vocab['house'], f'Test failed'
print('All tests passed!')

Now try to access the index of 'amsterdam' and compare it with the index of 'jakarta'. What does this teach us about the distribution of data and inclusion of global regions in US patents?

In [ ]:
#Keep cell

print(tokenizer_pat.vocab['amsterdam'])
#print(tokenizer_pat.vocab['jakarta']) gives KeyError

Finally as we have already seen, Hugging Face's Transformers assume that your target variable has the column name 'labels', but in our dataset it is called 'score'. Therefore, we need to rename it. There is a `rename_columns()` function to do that. Run the cell below.

In [ ]:
#Keep cell
ds_pat_encoded = ds_pat_encoded.rename_columns({'score':'labels'})

### Training, evaluation and test

Now that we have prepared our tokens and labels, we need to create our validation set. Kaggle often has a separate file for tests and validations.

Our downloaded data directory contains another file for the test data. Run the cell below to create `eval_df_pat`.

In [ ]:
#Keep cell

eval_df_pat = pd.read_csv('./uspppm-data/test.csv')
eval_df_pat.head(3)

As discussed, in machine learning we need to usually create distinct training, validation and test datasets. Earlier, we have skipped the validation dataset for demonstration purposes. So, let's show now how to add such a set.

The validation set is used to see how the model is doing during training, by, e.g., checking the model's predictions against the validation set at the end of an epoch. It is never used as input to training the model. The test dataset finally is only used at very end in order to evaluate the actual model. Please note that in the hate-speech example, we used the test dataset for the evaluation during training, as we did not have more data for a final test.

Let's use `train_test_split` to create a validation dataset. Remember, this is the Hugging Face version of the function and not to be confused with the one in sklearn. Check out https://huggingface.co/docs/datasets/en/process.

Run the cell below.

In [ ]:
#Keep cell
ds_pat_encoded_final = ds_pat_encoded.train_test_split(0.25, seed=42)
ds_pat_encoded_final

As said, the test dataset is not involved in the training at all and only used at the very end. In the case of Kaggle, we could submit the results from the test data to a leadership board to be ranked with our performance. But we skip this here.

We use `eval_df_pat` as our name for the test dataset in order to avoid confusion with the test dataset that was created by `train_test_split`. Run the cell below, which brings the steps above together to prepare the test data.

In [ ]:
#Keep cell

eval_df_pat['input'] = 'CON: ' + eval_df_pat['context'] + '; PHARSE1: ' + eval_df_pat['target'] + '; PHARSE2: ' + eval_df_pat['anchor']
eval_ds_pat = Dataset.from_pandas(eval_df_pat).map(tokenizer_pat_func, batched=True)

One more test whether you got it right.

In [ ]:
# Run this cell to test your code...
assert eval_df_pat.loc[15,'input']  == 'CON: G02; PHARSE1: illumination; PHARSE2: el display', f'Test failed'
print('All tests passed!')

### Training the model

Let's start training again and define our model with `model_pat = AutoModelForSequenceClassification.from_pretrained(model_pat, num_labels=2).to(device)`. Run the cell below, which also prints out the default device - just to check. Again, if you see 'cpu' here, you have to start from the beginning.

In [ ]:
#Keep cell

model_pat = AutoModelForSequenceClassification.from_pretrained(checkpoint_pat, num_labels=2).to(device)
print('Default device: ', device)

We are ready to start training again. For demonstration purposes, we will keep `compute_metrics()` and `training_args` from the Twitter example. This might not be a great idea in this case. We want to normally change the number of epochs or the learning rate for different data. But we want to mainly show you that the steps are very similar and are happy with slightly worse performance.

Run the cell below, which runs for quite a while depending on your hardware.

In [ ]:
#Keep cell

from transformers import Trainer
trainer = Trainer(model_pat,
                  training_args,
                  train_dataset=ds_pat_encoded_final['train'],
                  eval_dataset=ds_pat_encoded_final['test'],
                  processing_class=tokenizer_pat,
                  compute_metrics=compute_metrics)

trainer.train()

Nice performance.

Let's evaluate with the test dataset. Run the cell below, which brings together some of the steps from the hate-speech example. Do you recognize which ones?

In [ ]:
#Keep cell

np.argmax(trainer.predict(eval_ds_pat).predictions, axis=1)

We would need to investigate this further ...

But unfortunately, we have nothing to compare this with, as we simplified the Kaggle example into a binary classification. Also, if you read the instructions at Kaggle, we should have predicted the scores directly and not classes. But you are nevertheless happy, as you have just completed your second full Transformers implementation for a different dataset and understood the power of text to text. You are getting the hang of fine-tuning! This was not too hard, was it?

But it can be even easier, as we will find out next. We can skip all the fine-tuning and employ the models directly.

## A Tour of Transformer Applications

What we have done up to now is quite advanced. Let's take a step back and look a the Transformer models without further training/fine-tuning. We can use Hugging Face models directly to do many of the things we have seen in the lecture in a process called zero-shot learning (https://en.wikipedia.org/wiki/Zero-shot_learning).

Let's use as an example another state of the union. The text below `text_sotu_europe` is the beginning of the '2023 State of the Union Address by President von der Leyen' on the state of the European Union (https://ec.europa.eu/commission/presscorner/detail/ov/speech_23_4426).

In [ ]:
text_sotu_europe = """Honourable Members.
        In just under 300 days, Europeans will take to the polls in our unique and remarkable democracy.
        As with any election, it will be a time for people to reflect on the State of our Union and the work done by those that represent them.
        But it will also be a time to decide on what kind of future and what kind of Europe they want.
        Among them will be millions of first-time voters, the youngest of whom were born in 2008.
        As they stand in that polling booth, they will think about what matters to them.
        They will think about the war that rages at our borders. Or the impact of destructive climate change.
        About how artificial intelligence will influence their lives.
        Or of their chances of getting a house or a job in the years ahead.
        Our Union today reflects the vision of those who dreamt of a better future after World War II.
        A future in which a Union of nations, democracies and people would work together to share peace and prosperity.
        They believed that Europe was the answer to the call of history."""

There are many methods with this text that we can do with Transformers directly. We don't need fine-tuning or any more 'shots' to train. Let's start with a task that we are already familiar with.

### Text Classification

For text classification/sentiment analysis, we can simply run the cell below.

In [ ]:
#Keep cell

from transformers import pipeline

classifier = pipeline("text-classification")
classifier(text_sotu_europe)

The European state of the union speech makes a very positive impression, as you would expect.

### Named Entity Recognition

Named Entity Recognition is also easy with Transformers. Run the next cell.

In [ ]:
#Keep cell

ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text_sotu_europe)
pd.DataFrame(outputs)

And, we can do more. For instance, something called:

### Question Answering

Please run the cell below to answer the `question` about the input text.

In [ ]:
#Keep cell

reader = pipeline("question-answering")
question = "What does the Union reflect?"
outputs = reader(question=question, context=text_sotu_europe)
print('Answer: ', outputs['answer'])
pd.DataFrame([outputs])

Can we summarize the speech?

### Summarization

Of course, we can by running the next cell.

In [ ]:
#Keep cell

summarizer = pipeline("summarization")
outputs = summarizer(text_sotu_europe, max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

### Translation

We can also translate the text into German, the native language of the president. Run the next cell. As you can see, we load now a specific model in the `pipeline` command that is good at translating English into German. If we don't specific this, a default option is used. See whether you can identify them from this original source code: https://github.com/huggingface/transformers/blob/71688a8889c4df7dd6d90a65d895ccf4e33a1a56/src/transformers/pipelines.py.

In [ ]:
#Keep cell

translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text_sotu_europe, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

It's time to try it yourself. Change the next cell so that you use your own input text and see what the model thinks about its sentiment.

In [ ]:
#Keep cell
#Add your own text

your_text = "Your text goes here"

from transformers import pipeline

classifier = pipeline("text-classification")
classifier(your_text)

Maybe somebody wants to reply to the speech.

### Text Generation

Let's generate the reply text. Run the cell below and observe that we define a specific model for the pipeline. It is the Chat version of a very small Qwen model, open sourced by Alibaba.

In [ ]:
#Keep cell

model_name = 'Qwen/Qwen1.5-0.5B-Chat'

generator = pipeline("text-generation", model = model_name)
response = "Dear speaker, it is great to hear your vision for Europe."
prompt = text_sotu_europe + "\n\nResponse:\n" + response
outputs = generator(prompt, max_length=400)
print(outputs[0]['generated_text'])

It's a short but good reply, because we have not got much input text. But hopefully you can see the point.

With the patents, we have learned that we can map a lot of tasks like classification to become a text-to-text generation one. Let's see next how we can implement the classification of the European SOTU speech with QWEN chat. Everything has become a matter of the right kind of prompting. Take a look at the useful explanations at https://huggingface.co/docs/transformers/v4.43.2/tasks/prompting.

The next cell looks complicated but this is only because it simulates speaking to the chatbot - just like you can tell ChatGPT to tell you what the sentiment of the speech was. To this end, it uses a lot of the standard format of chatbots. Do you recognise which? And,  it uses powerful f-string and print methods in Python.

See whether you can understand it with the help of Gemini but definitely check out https://www.w3schools.com/python/python_string_formatting.asp. https://huggingface.co/blog/qwen-3-chat-template-deep-dive explains the QWEN chat template and why you need all those different parts.


Run the next cell.

In [ ]:
#Keep cell

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Format the prompt using the chat template
messages = [
    {"role": "system", "content": "You are a helpful assistant. Classify the sentiment of the provided text. Respond with only one word: 'positive', 'negative', or 'neutral'."},
    {"role": "user", "content": f"Text: \"{text_sotu_europe}\"\nSentiment:"}
]

# The tokenizer will apply the template and add the generation prompt for the assistant's turn
chat_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print(f"--- Chat Prompt Sent to Model ---\n{chat_prompt}\n------------------------------")

outputs = generator(
    chat_prompt,
    max_new_tokens=5,  # Just enough for "negative" or "neutral"
    pad_token_id=tokenizer.eos_token_id, # Important for some models to stop properly
    do_sample=False,
)

# Gemini explains:
# The output includes the prompt, so we need to extract only the generated part.
# The 'outputs' is a list (usually of one item unless num_return_sequences > 1).
# Your loop had a typo: `seq` should be `out`.
# for out in outputs:
#     print(f"Result: {out['generated_text']}")

generated_full_text = outputs[0]['generated_text']

# Extract only the newly generated part (the assistant's response)
# The chat_prompt already has the <|im_start|>assistant part if add_generation_prompt=True
# So, the generated text will be chat_prompt + assistant_response
# A simple way is to remove the original chat_prompt from the beginning of the generated_full_text
generated_response = generated_full_text[len(chat_prompt):].strip()

print(f"Full generated text sample: {generated_full_text[:500]}...") # Print a snippet
print(f"Extracted response: '{generated_response}'")

# Simple parsing
cleaned_response = "unknown"
cleaned_response = generated_response.lower().strip().replace(".", "")
#if cleaned_response in ["positive", "negative", "neutral"]:
#    final_sentiment = cleaned_response
#else:
    # Sometimes models add extra words, try to find the keyword
#    if "positive" in cleaned_response:
#        final_sentiment = "positive"
#    elif "negative" in cleaned_response:
#        final_sentiment = "negative"
#    elif "neutral" in cleaned_response:
#        final_sentiment = "neutral"

print(f"Parsed Sentiment: {cleaned_response}")

The latest is then to give the chatbots a few examples to learn from - in this case what is a positive, negative or neutral speech. This also called few-shot training or 'in-context learning': https://news.mit.edu/2023/large-language-models-in-context-learning-0207.

Run the next cell.

In [ ]:
# Keep cell

# --- Few-Shot Examples ---
# It's good to have a mix of sentiments
example_1_text = "I absolutely love this speech! It's fantastic and works perfectly."
example_1_sentiment = "positive"

example_2_text = "The weather today is just okay, nothing special to report."
example_2_sentiment = "neutral"

example_3_text = "I'm very disappointed with the politician's vision. It was a terrible speech."
example_3_sentiment = "negative"

# Gemini explains: Construct the Few-Shot Prompt using Chat Template ---
# The system prompt sets the overall instruction.
# The user prompt provides the examples and the text to classify.
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Your task is to classify the sentiment of the provided text. Please respond with only one word: 'positive', 'negative', or 'neutral'."
    },
    {
        "role": "user",
        "content": f"""
Here are some examples of how to classify sentiment:

Text: "{example_1_text}"
Sentiment: {example_1_sentiment}

Text: "{example_2_text}"
Sentiment: {example_2_sentiment}

Text: "{example_3_text}"
Sentiment: {example_3_sentiment}

Now, classify the following text:
Text: "{text_sotu_europe}"
Sentiment:"""
    }
]

chat_prompt_string = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("--- Full Chat Prompt Sent to Model ---")
print(chat_prompt_string)
print("--------------------------------------")

outputs = generator(
    chat_prompt_string,
    max_new_tokens=10,  # Should be enough for "positive", "negative", or "neutral"
    pad_token_id=tokenizer.eos_token_id,
)

# Since return_full_text=False, outputs[0]['generated_text'] is only the new text

generated_response = outputs[0]['generated_text'].strip()
print(f"\n-----------------------\nRaw generated response: '{generated_response}' \n--------------------")

# Simple parsing to get one of the target words
cleaned_response = "unknown"
cleaned_response = generated_response.lower().replace(".", "").replace("!", "").strip()
print(f"\nPredicted Sentiment: {cleaned_response}")


The Qwen example showcasing text generation and prompted classification highlights the growing capabilities of instruction-tuned models, which are becoming increasingly common. Remember the *DeepLearning.AI* Python with AI introduction? While the details can be a bit complex at first, it's enough for now to know that these models exist and that there's a lot of excitement around them. Few shot learning in particular is easy to implement and can lead to excellent results.

That's it! Your lab exercises will involve running some of the recent examples, creating a few of your own, and producing at least one fine-tuned Transformer for classification. Good luck!

### Save to html (uncomment to execute)

In [ ]:
# path_to_this_notebook = 'INSERT PATH TO THIS NOTEBOOK HERE'
# !jupyter nbconvert "{path_to_this_notebook}" --to html

Run the next cell to create an automated HTML download of your notebook for submission to Canvas. Please note that this might run for a while and **make sure you have saved the notebook** before running the code. If you asked to allow access to your Google Drive, please confirm.

In [ ]:
#Keep cell
import os

if not(os.path.exists('make_html.py')):
  print('Downloading make_html.py\n')
  !wget https://raw.githubusercontent.com/goto4711/cdai/refs/heads/main/make_html.py

from make_html import make_html

make_html("Week4_Workshop.ipynb") #Change the argument if your notebook is called differently